# Подход 1: градиентный бустинг "в лоб"

## Что указать в отчете
### В отчете по данному этапу вы должны ответить на следующие вопросы:

### Какие признаки имеют пропуски среди своих значений? Что могут означать пропуски в этих признаках (ответьте на этот вопрос для двух любых признаков)?
#### 'first_blood_time', 'first_blood_team', 'first_blood_player1', 'first_blood_player2', 'radiant_bottle_time', 'radiant_courier_time', 'radiant_flying_courier_time', 'radiant_first_ward_time', 'dire_bottle_time','dire_courier_time','dire_flying_courier_time','dire_first_ward_time'.
Признак first_blood_time - игровое время первой крови. Первая кровь не была пролита в течение первых 5 минут матча. Возможно обе команды были настроены на пассивную игру и попытки нарастить преимущество в лейте.
Признак radiant_first_ward_time - время установки командой первого "наблюдателя", т.е. предмета, который позволяет видеть часть игрового поля. Ни один из игроков команды dire не покупал "ward" в течение первых 5 минут матча. Игра без саппортов на низком рейтинге...

### Как называется столбец, содержащий целевую переменную?
radiant_win

### Как долго проводилась кросс-валидация для градиентного бустинга с 30 деревьями? Инструкцию по измерению времени можно найти ниже по тексту. Какое качество при этом получилось? Напомним, что в данном задании мы используем метрику качества AUC-ROC.
Затраченное время:  0:01:23, AUC = 0.69  

### Имеет ли смысл использовать больше 30 деревьев в градиентном бустинге? Что бы вы предложили делать, чтобы ускорить его обучение при увеличении количества деревьев?
С ростом числа деревьев, при сохранении остальных параметров увеличивается общее время расчета. Среднее значение AUC по кросс-валидации увеличивается при росте числа деревьев, что говорит нам о том, что использование большего числа деревьев увеличивает качество модели на обучающей выборке. Однако, с ростом качества на обучении увеличивается риск переобучения


## Загрузка обучающей и тестовой выборок

In [229]:
import pandas as pd

features = pd.read_csv('./features.csv', index_col='match_id')

features.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52.0,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5.0,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13.0,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27.0,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16.0,2449,0,4,1974,3,63


In [230]:
features_test = pd.read_csv('./features_test.csv', index_col='match_id')

features_test.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
6,1430287923,0,93,4,1103,1089,8,0,1,9,...,0,12.0,247.0,-86.0,272.0,3,4,2,0,118.0
7,1430293357,1,20,2,556,570,1,0,0,9,...,2,-29.0,168.0,-54.0,NaN,3,2,2,1,16.0
10,1430301774,1,112,2,751,808,1,0,0,13,...,1,-22.0,46.0,-87.0,186.0,1,3,3,0,-34.0
13,1430323933,1,27,3,708,903,1,1,1,11,...,2,-49.0,30.0,-89.0,210.0,3,4,2,1,-26.0
16,1430331112,1,39,4,1259,661,4,0,0,9,...,0,36.0,180.0,-86.0,180.0,1,3,2,1,-33.0


## Найдем признаки, отсутствующие в тестовой выборке.

In [231]:
itogs = [x for x in features.columns if x not in features_test.columns]

itogs

['duration',
 'radiant_win',
 'tower_status_radiant',
 'tower_status_dire',
 'barracks_status_radiant',
 'barracks_status_dire']

## Целевой признак - 'radiant_win'

In [232]:
y = features['radiant_win'].as_matrix()

y

array([1, 1, 0, ..., 0, 0, 1], dtype=int64)

## Оставим только те признаки, которые есть в тестовой выборке

In [233]:
features = features[[x for x in features.columns if x in features_test.columns]]

features.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,0,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,0,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,1,-39.0,45.0,-77.0,221.0,3,4,3,1,13.0
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,0,-30.0,124.0,-80.0,184.0,0,4,2,0,27.0
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,0,46.0,182.0,-80.0,225.0,6,3,3,0,-16.0


## Найдем все признаки в которых присутствуют пропуски

In [234]:
[k for k,v in dict(features.count() / features.shape[0]).items() if v < 1]

['first_blood_time',
 'first_blood_team',
 'first_blood_player1',
 'first_blood_player2',
 'radiant_bottle_time',
 'radiant_courier_time',
 'radiant_flying_courier_time',
 'radiant_first_ward_time',
 'dire_bottle_time',
 'dire_courier_time',
 'dire_flying_courier_time',
 'dire_first_ward_time']

## Возможная причина пропуска в данных:

### Признак first_blood_time - игровое время первой крови.

#### Первая кровь не была пролита в течение первых 5 минут матча. Возможно обе команды были настроены на пассивную игру и попытки нарастить преимущество в лейте.

### Признак radiant_first_ward_time - время установки командой первого "наблюдателя", т.е. предмета, который позволяет видеть часть игрового поля

#### Ни один из игроков команды dire не покупал "ward" в течение первых 5 минут матча. Игра без саппортов на низком рейтинге...

## Заменяем пропуски в обучающей выборке на 0 и проверяем, что не осталось ни одного признака с пропусками.

In [235]:
features = features.fillna(0)

[k for k,v in dict(features.count() / features.shape[0]).items() if v < 1]

[]

## Обучение модели градиентного бустинга

In [243]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
import numpy as np
from sklearn.model_selection import KFold, cross_val_score

kf = KFold(n_splits=5, shuffle=True, random_state=42)

X = features.as_matrix()

for n in [10,20,30,40,50,60, 70, 80, 90, 100]:
    
    start_time = datetime.datetime.now()
    
    gbc = GradientBoostingClassifier(n_estimators=n, verbose=False, random_state=42)
    
    model_scores = cross_val_score(gbc, X, y, cv=kf, scoring='roc_auc')
        
    print("Деревьев: ", n, " Среднее значение AUC: ", round(np.mean(model_scores),4), " Затраченное время: ", datetime.datetime.now() - start_time)

Деревьев:  10  Среднее значение AUC:  0.6649  Затраченное время:  0:00:30.226027
Деревьев:  20  Среднее значение AUC:  0.6825  Затраченное время:  0:00:57.912176
Деревьев:  30  Среднее значение AUC:  0.69  Затраченное время:  0:01:23.821708
Деревьев:  40  Среднее значение AUC:  0.694  Затраченное время:  0:02:18.935141
Деревьев:  50  Среднее значение AUC:  0.6975  Затраченное время:  0:03:47.435787
Деревьев:  60  Среднее значение AUC:  0.7001  Затраченное время:  0:03:43.914903
Деревьев:  70  Среднее значение AUC:  0.7021  Затраченное время:  0:03:07.210351
Деревьев:  80  Среднее значение AUC:  0.7038  Затраченное время:  0:03:31.174732
Деревьев:  90  Среднее значение AUC:  0.7052  Затраченное время:  0:03:55.950579
Деревьев:  100  Среднее значение AUC:  0.7062  Затраченное время:  0:04:36.885805


## С ростом числа деревьев, при сохранении остальных параметров увеличивается общее время расчета. Среднее значение AUC по кросс-валидации увеличивается при росте числа деревьев, что говорит нам о том, что использование большего числа деревьев увеличивает качество модели на обучающей выборке. Однако, с ростом качества на обучении увеличивается риск переобучения

# Подход 2: логистическая регрессия

## Что указать в отчете
### В отчете по данному этапу вы должны ответить на следующие вопросы:

### Какое качество получилось у логистической регрессии над всеми исходными признаками? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?
Лучшее значение AUC получилось 0.7166 при С = 0.01. На модели бустинга подобное значение качества не было достигнуто при количестве деревьев меньше 100. Логистическая регрессия значительно быстрее бустинга.

### Как влияет на качество логистической регрессии удаление категориальных признаков (укажите новое значение метрики качества)? Чем вы можете объяснить это изменение?
Их удаление практически не повлияло на итоговые результаты, что говорит о низком влиянии данных показателей на модель. Лучшее значение AUC получилось 0.7166 при С = 0.01.

### Сколько различных идентификаторов героев существует в данной игре?
112

### Какое получилось качество при добавлении "мешка слов" по героям? Улучшилось ли оно по сравнению с предыдущим вариантом? Чем вы можете это объяснить?
Применение "мешка слов" привело к улучшению результатов модели на кросс-валидации. Лучшее значение AUC получилось 0.7519   при С = 0.1, что может быть объяснено включением в модель информации о составе каждоый из команд, что определенно должно повлиять на результаты матчей

### Какое минимальное и максимальное значение прогноза на тестовой выборке получилось у лучшего из алгоритмов?
минимум = 0.0086, максимум = 0.9965


In [241]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

scaler = StandardScaler()

X = scaler.fit_transform(features.as_matrix())

for C in [10.0 ** i for i in range(-5,5)]:
    
    start_time = datetime.datetime.now()
    
    lr = LogisticRegression(C=C, random_state=42, penalty='l2')
    
    model_scores = cross_val_score(lr, X, y, cv=kf, scoring='roc_auc')
        
    print("C: ", C, " Среднее значение AUC: ", round(np.mean(model_scores),4), " Затраченное время: ", datetime.datetime.now() - start_time)

C:  1e-05  Среднее значение AUC:  0.6952  Затраченное время:  0:00:02.948323
C:  0.0001  Среднее значение AUC:  0.7114  Затраченное время:  0:00:04.840465
C:  0.001  Среднее значение AUC:  0.7164  Затраченное время:  0:00:08.840887
C:  0.01  Среднее значение AUC:  0.7166  Затраченное время:  0:00:11.191529
C:  0.1  Среднее значение AUC:  0.7165  Затраченное время:  0:00:13.025532
C:  1.0  Среднее значение AUC:  0.7165  Затраченное время:  0:00:13.012543
C:  10.0  Среднее значение AUC:  0.7165  Затраченное время:  0:00:12.617107
C:  100.0  Среднее значение AUC:  0.7165  Затраченное время:  0:00:12.392449
C:  1000.0  Среднее значение AUC:  0.7165  Затраченное время:  0:00:12.346518
C:  10000.0  Среднее значение AUC:  0.7165  Затраченное время:  0:00:12.547783


## Удалим категориальные переменные

In [246]:
X = features[[x for x in features.columns if x not in ['lobby_type', 'r1_hero', 'r2_hero', 
                                                       'r3_hero','r4_hero','r5_hero','d1_hero', 
                                                       'd2_hero', 'd3_hero','d4_hero','d5_hero']]]

X = scaler.fit_transform(X.as_matrix())

for C in [10.0 ** i for i in range(-5,5)]:
    
    start_time = datetime.datetime.now()
    
    lr = LogisticRegression(C=C, random_state=42, penalty='l2')
    
    model_scores = cross_val_score(lr, X, y, cv=kf, scoring='roc_auc')
        
    print("C: ", C, " Среднее значение AUC: ", round(np.mean(model_scores),4), " Затраченное время: ", datetime.datetime.now() - start_time)

C:  1e-05  Среднее значение AUC:  0.6951  Затраченное время:  0:00:02.706441
C:  0.0001  Среднее значение AUC:  0.7113  Затраченное время:  0:00:04.324435
C:  0.001  Среднее значение AUC:  0.7164  Затраченное время:  0:00:08.095685
C:  0.01  Среднее значение AUC:  0.7166  Затраченное время:  0:00:10.183512
C:  0.1  Среднее значение AUC:  0.7165  Затраченное время:  0:00:11.189716
C:  1.0  Среднее значение AUC:  0.7165  Затраченное время:  0:00:12.417006
C:  10.0  Среднее значение AUC:  0.7165  Затраченное время:  0:00:12.689146
C:  100.0  Среднее значение AUC:  0.7165  Затраченное время:  0:00:12.441751
C:  1000.0  Среднее значение AUC:  0.7165  Затраченное время:  0:00:12.257701
C:  10000.0  Среднее значение AUC:  0.7165  Затраченное время:  0:00:12.245718


## Добавим "мешок слов"

In [252]:
heroes = pd.read_csv('./data/dictionaries/heroes.csv')
print('Всего героев в игре:', len(heroes))

Всего героев в игре: 112


In [254]:
N = len(heroes)

X_pick = np.zeros((features.shape[0], N))

for i, match_id in enumerate(features.index):
    
    for p in range(5):
        
        X_pick[i, features.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, features.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1
        
X_pick

C:\Users\ccord517\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  if __name__ == '__main__':


array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0., -1.]])

In [272]:
X = pd.concat([pd.DataFrame(X, index=features.index), pd.DataFrame(X_pick, index=features.index)], axis=1).as_matrix()

X

array([[-2.54436416,  1.40080818,  1.52597175, ...,  0.        ,
         0.        ,  0.        ],
       [-2.54045236,  0.50131354, -0.08013929, ...,  0.        ,
         0.        ,  0.        ],
       [-2.53923104,  0.50131354,  0.1510701 , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 1.09874571,  0.50131354,  0.29226667, ...,  0.        ,
         0.        ,  1.        ],
       [ 1.09895204, -0.39818111, -0.17368203, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.1026479 , -0.39818111, -0.31840851, ...,  0.        ,
         0.        , -1.        ]])

In [276]:
for C in [10.0 ** i for i in range(-5,5)]:
    
    start_time = datetime.datetime.now()
    
    lr = LogisticRegression(C=C, random_state=42, penalty='l2')
    
    model_scores = cross_val_score(lr, X, y, cv=kf, scoring='roc_auc')
        
    print("C: ", C, " Среднее значение AUC: ", round(np.mean(model_scores),4), " Затраченное время: ", datetime.datetime.now() - start_time)

C:  1e-05  Среднее значение AUC:  0.6992  Затраченное время:  0:00:03.793696
C:  0.0001  Среднее значение AUC:  0.7251  Затраченное время:  0:00:05.647936
C:  0.001  Среднее значение AUC:  0.7463  Затраченное время:  0:00:10.390910
C:  0.01  Среднее значение AUC:  0.7517  Затраченное время:  0:00:16.175406
C:  0.1  Среднее значение AUC:  0.7519  Затраченное время:  0:00:22.302058
C:  1.0  Среднее значение AUC:  0.7519  Затраченное время:  0:00:23.056440
C:  10.0  Среднее значение AUC:  0.7519  Затраченное время:  0:00:23.079859
C:  100.0  Среднее значение AUC:  0.7519  Затраченное время:  0:00:24.099362
C:  1000.0  Среднее значение AUC:  0.7519  Затраченное время:  0:00:23.945394
C:  10000.0  Среднее значение AUC:  0.7519  Затраченное время:  0:00:23.444619


## Предскажем вероятность победы radiant на тестовой выборке по лучшей модели

In [277]:
lr = LogisticRegression(C=0.1, random_state=42, penalty='l2')

lr.fit(X,y)

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [303]:
features_test = features_test.fillna(0)

X_test = features_test[[x for x in features_test.columns if x not in ['lobby_type', 'r1_hero', 'r2_hero', 
                                                       'r3_hero','r4_hero','r5_hero','d1_hero', 
                                                       'd2_hero', 'd3_hero','d4_hero','d5_hero']]]

X_test = scaler.fit_transform(X_test.as_matrix())

X_pick_test = np.zeros((features_test.shape[0], N))

for i, match_id in enumerate(features_test.index):
    
    for p in range(5):
        
        X_pick_test[i, features_test.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick_test[i, features_test.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1
        
X_test = pd.concat([pd.DataFrame(X_test, index=features_test.index), pd.DataFrame(X_pick_test, index=features_test.index)], axis=1).as_matrix()

X_test

C:\Users\ccord517\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  from ipykernel import kernelapp as app


array([[-2.5148749 ,  0.47474566, -0.23774298, ...,  0.        ,
         0.        ,  0.        ],
       [-2.51389501, -1.36563036, -1.20791332, ...,  0.        ,
         0.        ,  0.        ],
       [-2.5123772 , -1.36563036, -0.86205735, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 1.08005567, -1.36563036, -1.17421453, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.08387463,  0.47474566,  1.0765097 , ...,  0.        ,
         0.        , -1.        ],
       [ 1.08579655,  0.47474566, -0.0390975 , ...,  0.        ,
         0.        ,  1.        ]])

In [314]:
pred = lr.predict_proba(X_test)[:,1]

pd.concat([pd.DataFrame(X_test, index=features_test.index), pd.DataFrame({'radiant_win ':pred}, index=features_test.index)], axis=1)

,0,1,2,3,4,5,6,7,8,9,...,194,195,196,197,198,199,200,201,202,radiant_win
match_id,,,,,,,,,,,,,,,,,,,,,
6,-2.514875,0.474746,-0.237743,-0.137094,-0.370005,-0.532732,1.003884,0.305165,-0.368985,0.003979,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.824665
7,-2.513895,-1.365630,-1.207913,-1.255660,-1.145090,-0.532732,-0.593200,0.305165,0.567766,0.023887,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.757165
10,-2.512377,-1.365630,-0.862057,-0.742715,-1.145090,-0.532732,-0.593200,1.898546,-1.305736,-1.375081,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.187787
13,-2.508381,-0.445442,-0.938323,-0.537967,-1.145090,0.968865,1.003884,1.101855,-1.305736,-0.920823,...,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.862486
16,-2.507087,0.474746,0.038942,-1.059534,-0.812911,-0.532732,-0.593200,0.305165,1.504517,0.945070,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.239917
18,-2.506518,1.394934,1.472027,0.854314,0.405079,-0.532732,-0.593200,-0.093180,0.567766,0.139714,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.379129
19,-2.506387,0.474746,1.108435,0.720690,1.401616,-0.532732,1.003884,-1.288215,-0.368985,-0.519050,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.528699
24,-2.503777,1.394934,1.337232,1.789686,0.847984,2.470463,-0.593200,0.703510,0.567766,0.574063,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.565878
33,-2.500708,-1.365630,-1.277085,-1.406527,-1.255816,-0.532732,1.003884,0.305165,0.567766,0.731515,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.215929


In [318]:
round(min(pred),4), round(max(pred),4)

(0.0086, 0.9965)